In [ ]:
!pip install twder
!pip install yfinance
!pip install pandas

In [ ]:
import yfinance as yf
import requests
import pandas as pd

stocks = ["TSLA", "MSFT", "0700.HK", "1398.HK"]
data = {}

for stock in stocks:
    stock_data = yf.Ticker(stock)
    stock_info = stock_data.info
    currency = stock_info.get("currency", "USD")
    price_data = stock_data.history(period="1mo")["Close"]
    data[stock] = {"prices": price_data, "currency": currency}

def get_exchange_rate():
    url = "https://api.exchangerate-api.com/v4/latest/USD"
    response = requests.get(url).json()
    return response['rates']['TWD']

usd_to_twd = get_exchange_rate()

def get_hkd_to_twd_rate():
    url = "https://api.exchangerate-api.com/v4/latest/USD"
    response = requests.get(url).json()
    usd_to_hkd = response['rates']['HKD']
    usd_to_twd = response['rates']['TWD']
    return usd_to_twd / usd_to_hkd

hkd_to_twd = get_hkd_to_twd_rate()


for stock, info in data.items():
    if info['currency'] == "USD":
        data[stock]["price_twd"] = info['prices'] * usd_to_twd
    elif info['currency'] == "HKD":
        data[stock]["price_twd"] = info['prices'] * hkd_to_twd

dfs = {}
for stock, info in data.items():
    dfs[stock] = pd.DataFrame({
        '股價(USD)': info['prices'],
        '股價 (TWD)': info['price_twd']
    })

final_df = pd.concat(dfs).rename_axis(['公司', '日期']).reset_index()
print(final_df)
final_df.to_csv("stock_prices_in_twd.csv", index=False)

         公司                         日期     股價(USD)      股價 (TWD)
0      TSLA  2024-11-18 00:00:00-05:00  338.739990  11009.049683
1      TSLA  2024-11-19 00:00:00-05:00  346.000000  11245.000000
2      TSLA  2024-11-20 00:00:00-05:00  342.029999  11115.974960
3      TSLA  2024-11-21 00:00:00-05:00  339.640015  11038.300476
4      TSLA  2024-11-22 00:00:00-05:00  352.559998  11458.199921
..      ...                        ...         ...           ...
83  1398.HK  2024-12-12 00:00:00+08:00    4.840000     20.244531
84  1398.HK  2024-12-13 00:00:00+08:00    4.780000     19.993566
85  1398.HK  2024-12-16 00:00:00+08:00    4.860000     20.328186
86  1398.HK  2024-12-17 00:00:00+08:00    4.850000     20.286357
87  1398.HK  2024-12-18 00:00:00+08:00    4.860000     20.328186

[88 rows x 4 columns]
